# Correcting Datacube Transit Times

When separating datacube files in AstroImageJ, the times in the series are not preserved. Instead, each slice inherits the same FITS header as the first slice. This means that the time of the first slice is used for all slices in the series. To correct this, we need to use the initial times of each datacube, the exposure time, and the frame transfer time to calculate the correct time for each slice.

In each fits header, I also corrected the dimensionality of the data by removing NAXIS3, as each slice is two dimensional. Finally, I fixed the filenames to have only 1 .fits ending.

In [ ]:
# loading required packages
from astropy.io import fits
from pathlib import Path
import os
import datetime

In [ ]:
# loops over all raw files, i is kept as a counter to keep track of the slice number
i = 0

# make sure to only use .fits files
paths = sorted(Path('raw').glob('*.fits'))

for file_name in paths:
    # open file
    print("currently trying:" + str(file_name))
    fits_inf = fits.open(str(file_name), ignore_missing_simple=True)
    header = fits_inf[0].header

    # remove NAXIS3, rename FRAME to DATE-OBS, DATE-OBS to USER-START
    header.pop('NAXIS3', None)
    header['STR-TIME'] = header.pop('DATE-OBS', None)
    header['DATE-OBS'] = header.pop('FRAME', None)

    # get info, calculate slice time, edit date modified
    time_between_slices = datetime.timedelta(seconds=header['ACT'])
    initial_time = datetime.datetime.strptime(header['DATE-OBS'],"%Y-%m-%dT%H:%M:%S.%f")
    slice_time = initial_time + i * time_between_slices
    slice_time_str = slice_time.strftime("%Y-%m-%dT%H:%M:%S.%f")
    date_modified = datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")

    # update header with times
    header['DATE-OBS'] = slice_time_str
    header['IRAF-TLM'] = date_modified

    output_dir = Path('time_corrected')
    output_dir.mkdir(exist_ok=True)

    # save the corrected FITS file
    p = str(file_name).split('/')[1]
    # for windows, use below
    # p = str(file_name).split('\\')[1]

    p = p.split('.')
    new_file_name = p[0] + "." + p[1] + '_timecorr.fits'
    output_path = 'time_corrected/' + new_file_name

    # save the corrected FITS file
    fits_inf.writeto(output_path, overwrite=True)
    print(f"Processed and saved: {output_path}")

    i+=1